# Westeros Tutorial - Introducing share constraints

In this tutorial, we show how to add *share constraints* to a MESSAGEix scenario. The share constraints are aimed at limiting the activity of a single or set of technologies to a certain share of the activity of another single or set of technologies.

This is a very useful feature for policy analysis. For example, in many of the nationally determined contributions (NDCs) as part of the Paris Agreement, numerous countries formulated part of their targets in terms of *shares*. The European Union, for example, has comitted to a target where renewables will make up 30% of the energy mix by 2030. How to implement such a policy in an energy model will be the focus of this tutorial.

**Pre-requisites**
- You have the *MESSAGEix* framework installed and working
- You have run Westeros baseline scenario (``westeros_baseline.ipynb``) and solved it successfully

## Adding a minimum share constraint
Building on the '*baseline*' scenario, we add a share constraint assuming that Westeros has set itself a target to generate 50% of its electricity from renewable energy by 720.

In [ ]:
import ixmp

mp = ixmp.Platform()

## Making a clone of the existing scenario 'baseline'

Loading the existing scenario '*baseline*' and cloning to a new scenario '*share_constraint*' to add share constraints.

In [ ]:
import message_ix

model = "Westeros Electrified"
base = message_ix.Scenario(mp, model=model, scenario="baseline")
scen = base.clone(
    model,
    "share_constraint",
    "illustration of share-constraint formulation",
    keep_solution=False,
)
scen.check_out()

Retrieving required information for constructing input data of share parameters.

In [ ]:
country = "Westeros"

## Steps required to add share constraints
The following steps are required in order to introduce a share constraint:
1. Define a new 'share' and add this to the *set* `"shares"`
2. Define which technologies contribute towards the total
3. Define which technologies contribute towards fulfilling the share
4. Define the share for relevant periods

The implementation of shares in MESSAGEix is generic and flexible, so that any combination of commodities, levels, technologies and nodes can be put in relation to any other combination.

### 1. Defining a new "share" 
We define a new share called `"share_renewable_electricity"`, and add this to the *set* `"shares"`.

In [ ]:
shares = "share_renewable_electricity"
scen.add_set("shares", shares)

### 2. Defining technologies that make up the total
We need to define which technologies make up the group accounting for the total, to which the share applies. To do this, we need to map the share to the relevant *commodity* and *level*.

The aim is to increase the share of electricity generated from renewables to 50% in 720. Therefore, we need to define which technologies contribute to total electricity generation at the secondary energy level. In our example, these are `"wind_ppl"` and `"coal_ppl"` technologies, which will be grouped as a new `type_tec`, called `"electricity_total"` in the scenario. For this `type_tec`, we need to define the relevant node, mode, level and commodity through the *set* `"map_shares_commodity_total"`.

In [ ]:
type_tec = "electricity_total"
scen.add_cat("technology", type_tec, "wind_ppl")
scen.add_cat("technology", type_tec, "coal_ppl")

For adding data to a MESSAGEix set or parameter, the index names must be correctly specified. To see the index names of a set or parameter, we can use the method: `scenario.idx_names()`.

In [ ]:
# Example of index names of a set
scen.idx_names("map_shares_commodity_total")

In [ ]:
import pandas as pd

# Adding data to a set using index names
df = pd.DataFrame(
    {
        "shares": [shares],
        "node_share": country,
        "node": country,
        "type_tec": type_tec,
        "mode": "standard",
        "commodity": "electricity",
        "level": "secondary",
    }
)
scen.add_set("map_shares_commodity_total", df)

### 3. Defining technologies of share
Next, we define technologies that contribute towards fulfilling the share. We also need to set the corresponding commodity and level for which the share constraint applies. In our example, the technology that provides 50% electricity from renewable energy is `"wind_ppl"`.

In [ ]:
type_tec = "electricity_renewable"
scen.add_cat("technology", type_tec, "wind_ppl")

# Reminder: use <scen.idx_names('map_shares_commodity_share')> to see the index names
df = pd.DataFrame(
    {
        "shares": [shares],
        "node_share": country,
        "node": country,
        "type_tec": type_tec,
        "mode": "standard",
        "commodity": "electricity",
        "level": "secondary",
    }
)
scen.add_set("map_shares_commodity_share", df)

### 4. Defining the share
Lastly, the actual share value needs to be added to the model. To represent a minimum share of 50% of total electricity generation from renewables, we use the parameter `"share_commodity_lo"`. If this constraint should act as an upper bound, then the parameter `"share_commodity_up"` can be used.

In [ ]:
# Fetching the index names of a parameter
scen.idx_names("share_commodity_lo")

In [ ]:
# Adding data to the parameter
df = pd.DataFrame(
    {
        "shares": shares,
        "node_share": country,
        "year_act": [720],
        "time": "year",
        "value": [0.5],
        "unit": "-",
    }
)
scen.add_par("share_commodity_lo", df)

### Commit and solve

In [ ]:
scen.commit(comment="Define parameters for minimum renewable share constraint")
scen.set_as_default()

In [ ]:
scen.solve()

In [ ]:
scen.var("OBJ")["lvl"]

# Plotting Results

In [ ]:
from message_ix.report import Reporter
from message_ix.util.tutorial import prepare_plots

%matplotlib inline

# Scenaio without share constraint
base_rep = Reporter.from_scenario(base)
prepare_plots(base_rep)

# Scenario with share constraint
rep = Reporter.from_scenario(scen)
prepare_plots(rep)

## Activity
***
In the new scenario ('*share_constraint*'), the activity of `"wind_ppl"` accounts for 50% of total electricity generation in 720. This was the share constraint we added to the scenario.

### Question:
The results show that the activity of `"wind_ppl"` in 710 is higher than in the '*baseline*' scenario. Do you know why this happens?

### Scenario: '*baseline*'

In [ ]:
plants = ["coal_ppl", "wind_ppl"]
base_rep.set_filters(t=plants)
base_rep.get("plot activity")

### Scenario: '*share_constraint*'

In [ ]:
rep.set_filters(t=plants)
rep.get("plot activity")

### Response to the question: 
This is because `"wind_ppl"` has a diffusion constraint in this scenario, limiting the activity growth rate to 10% per year. Thus, to achieve the 50% target in 720 specified by the share constraint, additional activity is required in the previous periods to ramp up the wind generation.

## Capacity
***
There is a big increase in 720 regarding activity which is reflected in the capacity for the two technologies.

### Scenario: '*baseline*'

In [ ]:
base_rep.get("plot capacity")

### Scenario: '*share_constraint*'

In [ ]:
rep.get("plot capacity")

## Prices
***
As expected, electricity prices in 720 have also increased vis-a-vis the '*baseline*'.

### Scenario: '*baseline*'

In [ ]:
filters = dict(t=None, c=["light"])
base_rep.set_filters(**filters)
base_rep.get("plot prices")

### Scenario: '*share_constraint*'

In [ ]:
rep.set_filters(**filters)
rep.get("plot prices")

In [ ]:
mp.close_db()